In [2]:
# This is the function version of the main, outputting a given amount of the overall values calculated. 

#POWER MODEL MAIN
# Created: 10/06/2022
# Author: Cameron King
# Function Authors: Henri Wessels, Alec Church

# Description:
#
# This is the main script that models the power collection, transmission, and receiving systems/processes.
# Generally takes in satellite design choices and varies altitude and transmission time to see what orbits/transmission
# times are possible with current choices, and which of those are most efficient. 
# These outputs of most efficient range of orbit/time, as well as sub-models made during the formation of this model, 
# will be given to other groups within the team to inform on their design decisions as well as ROI, mass, MOI, etc.


# libraries
import math
import numpy
import mpmath
import matplotlib.pyplot as plt

# functions
from ipynb.fs.full.efficiency_funcs import *
from ipynb.fs.full.transmission_funcs import *
from ipynb.fs.full.GaussNCone import *
from ipynb.fs.full.Current_Orbit_Values import *

In [3]:
def PowerMain_func( h, V, Period, eclipse_percent,   panelSize,   LI_battery_mass_total,   laser_intake_wattage, r_aperture,    r  ):
# determines the total energy recieved and the total efficiency of transmission

    # attitude & position errors - keep these at 0 unless analyzing error effects
    pos_err = [0,0,0]
    point_err = [0,0]

#####################################################################################
################################   Constants    #####################################
#####################################################################################

    # moon
    mu_m = 4.905E12; # Gravitational parameter of the moon, 
    r_m = 1737500; # radius of the moon, m

    # battery & pane constants
    satLife = float(10); # years
    degPYear = float(0.01); # 1% 
    thetaMax = float(0); # informs peak power production
    I_d = float(0.77); # inherent degradation (0.49-0.88)----------SMAD
    BOLEff = float(0.3); #Beginning of Life Efficiency, 30 % ----- https://www.nasa.gov/smallsat-institute/sst-soa/power 
    BOLPmp = float(400); # W/m^2 ----------------------------------https://www.nasa.gov/smallsat-institute/sst-soa/power 
    specPow = float(100); # W/kg ----------------------------------https://www.nasa.gov/smallsat-institute/sst-soa/power
    DoD = 0.4; # Depth pf Discharge
    LI_battery_upperBound = 0.15; # Battery can't allocate this capacity to anything else
    LI_battery_lowerBound = 0.15; # Battery can't allocate this capacity to anything else
    bounds = LI_battery_upperBound + LI_battery_lowerBound # total sum of the bounds
    SatSurvival = 0.05; # Battery dedicated to onboard computing
    LI_EOL_const = 0.85;  #0.85 is from EOL modeling
    Panel_Eff = 0.32 # solar panel efficiency within the assumed range of +/- 22.5 Degrees
    theta_panel = float(0.4); # Influences cosine loss 22.5 deg worst case -> 0.4 rad


    P_per_kg = 1500 # W/kg
    E_per_kg = 200 # Wh/kg 

    # C = 7.5 -> relatively healthy discharge rate
    # battery power and energy density
    # Battery Type
        # specs : https://www.nasa.gov/smallsat-institute/sst-soa/power --> table 3-7
        # Lithium ion -> Power Density: 1500 W/kg
        #                Gravimetric Energy: 100-265 Wh/kg
        # BATTERY CYCLING SOURCE
        # https://iopscience.iop.org/article/10.1149/1945-7111/abf05f/pdf 

    # Comms
    Comm_Power = 100 # watts of constant power draw for comms system
    
    # laser
    laser_loss = 0.55 # percentage of loss of power in the laser itself
    
    # receiver
    rec_zleff = 0.30 # receiver's zero loss efficiency (normal / maximum efficiency of flux power to elec power) 
    rec_b_0 = 0.1 # reflectivity constant, 0.1 for 1 sheet of glass, 0.2 for 2
    rec_I_cutoff = 1380*450 # W/m^2 Max flux receiver can withstand, any higher flux and this is the power accepted. This caps the flux allowed.
    
#####################################################################################
#####################################################################################
#####################################################################################

    Feasible = 1;

###### Calculate power generated by the solar panels ######

    L_d = (1-degPYear)**satLife; # % (How much the satellite degrades over a lifetime)

    P_eol = Panel_Eff* BOLPmp * L_d * math.cos(theta_panel); # Specific power at end of life
    P_0 = P_eol*panelSize; # power available at end of life, assume this is the power available during the whole lifetime

    P_0_noComm = P_0 - Comm_Power

###### Battery losses and allocation ######

    LI_usablebattery_mass = LI_battery_mass_total * 0.5; # Redundancy: Makes sure that there is a secondary battery system if the first fails for some reason

    LI_battery_capacity_total = LI_usablebattery_mass*E_per_kg* LI_EOL_const *(1-SatSurvival-bounds); # [Wh] # same assumption of end of life power output of panels; battery at end of life has LI_EOL_const amount of initial value,
    LI_battery_discharge = LI_usablebattery_mass*P_per_kg * LI_EOL_const;     # [W]  # so entire lifetime we assume we are operating at end of life conditions

    LI_battery_capacity_laser = DoD*LI_battery_capacity_total # energy capacity for the laser

    # feasibility checks

    if LI_battery_discharge < laser_intake_wattage: 
        Feasible = 0; # not enough mass of batt to power high wattage laser

    #Satellite can charge panels in half an orbital period
    E2Batt = P_0_noComm * Period * (1-eclipse_percent); # [Wh] Assume battery charges for half of the orbit period
    if E2Batt < LI_battery_capacity_total:
        Feasible = 0;    


###### calculate max transmission time for current battery specs & receiver specs ######


    # laser loss and maximum discharge time
    L_W = laser_intake_wattage*(1-laser_loss) # Laser Wattage, this is the battery/capaciter AVERAGE watt output possible, minus the power loss of the laser
    t_max_battery = LI_battery_capacity_laser/laser_intake_wattage*3600; # max discharge time, equal to maximum transmission time for this battery

    # receiver maximum trasnmission time given receiver efficiency, this takes time of total view into account
    theta_r_max = mpmath.acos(rec_b_0/(1+rec_b_0))
    theta_s_max_receiver = mpmath.asin( r_m* mpmath.sin(numpy.pi-theta_r_max)/(r_m+h) )
    alpha_max_receiver = theta_r_max-theta_s_max_receiver
    t_max_receiver = 2*(r_m+h)*alpha_max_receiver/V

    t_end = min([t_max_battery,t_max_receiver]) # choose the smallest maximum time possible with given orbit & battery 


###### do one pass simulation, calculate orbit averages and focal length to define beam conditions ######


    # time step accuracy
    t_step = t_end/1000; # size of time step, s
    N = t_end/t_step; # number of elements in every model array ALWAYS USE THIS FOR LOOPS
    N = int(N)

    # preallocate
    t = numpy.zeros(N);
    d = numpy.zeros(N);
    theta_s = numpy.zeros(N);
    theta_r = numpy.zeros(N);
    FOV = numpy.zeros(N);
    r_prime = numpy.zeros(N);
    dtheta_s = numpy.zeros(N);
    dtheta_s_approx = numpy.zeros(N);
    ddtheta_s_approx = numpy.zeros(N);
    # loop through time period to figure out average distance and average size of the receiver
    for i in range(0,N):
        t[i] = i*t_step; # calculate current time
        
        current_sich = Current_Orbit_Values(h,t_end,t[i],V,r)
        
        # split up the output from Current_Orbit_Values into useful values to save
        d[i] = current_sich[0]
        theta_s[i] = current_sich[1]
        theta_r[i] = current_sich[2]
        FOV[i] = current_sich[3]
        r_prime[i] = current_sich[4]
        
    # find pointing speed, approximatly
    if i == 0 :
        dtheta_s_approx[i] = 0
        ddtheta_s_approx[i] = 0
    else :
        dtheta_s_approx[i] = abs(theta_s[i] - theta_s[i-1])
        ddtheta_s_approx[i] = dtheta_s_approx[i] - dtheta_s_approx[i-1]
        
    
    # calculate average distance and r_prime for part 3, and find focal length

    d_ave = numpy.mean(d)
    r_b = numpy.mean(r_prime)

    alpha_ave = mpmath.atan((r_b-r_aperture)/d_ave) # ideal view angle, this is the defining angle of the shape of the beam
    focal_length = -r_aperture/mpmath.tan(alpha_ave) # focal length of necessary diverging lens, neg cause diverging
    
###### using defined beam conditions, simulate the beam at every point on reciever during transmission ######

    # preallocations
    F_disp = [];
    P_T = [];
    UA_F_disp = [];
    I_ave = numpy.zeros([N-1,100]);
    I_max = numpy.zeros(N-1)
    
    # loop through the transmission period 
    # this section also accounts for the max intensity, but also measures the UA = UNADJUSTED flux dispursion to determin soley position error down the line
    for i in range(0,N-1):
    
        current_disp = gaussNcone_transmission_func(r_aperture,r_b,d_ave,d[i],L_W)
        current_disp = numpy.array(current_disp)
    
        # check if intensity of shell is above the maximum, adjust the percentage within to keep I_ave[shell] < rec_I_cutoff
        for j in range(1,len(current_disp[1,:])):
            P_perc_old = current_disp[1,j]
            A_shell = (numpy.pi*(current_disp[0,j]**2-current_disp[0,j-1]**2)) # area of the shell rn
            I_ave[i,j] = (current_disp[1,0]*current_disp[1,j])/A_shell # (total power * percent of power within shell) / area of shell
    
            if I_ave[i,j] >= rec_I_cutoff: # we need to reassign the second row of current_disp to rec_I_cutoff = P_within / A_shell
                P_allowed = rec_I_cutoff*A_shell
                P_perc_new = P_allowed/current_disp[1,0]
                current_disp[1,j] = P_perc_new
    
        I_max[i] = max(I_ave[i,:])  
        F_disp.append(current_disp) 
        P_T.append(current_disp[1,0])

    F_disp = numpy.array(F_disp) # flux dispursion for each time step in a matrix
    UA_F_disp = numpy.array(UA_F_disp) # this is the incident flux, unadjusted (UA) for the receiver's max intensity
    
    
###### Using the flux dispursion at every time step, determine efficiency at every step, and in total ######
    
    # preallocations
    n_rec = numpy.zeros(N);
    n_pos = numpy.zeros(N);
    UA_n_pos = numpy.zeros(N);
    E_R = numpy.zeros(N);
    E_T = numpy.zeros(N);
    
    for i in range(0,N-1):

        # this function calculates the efficiency associated with incidence angle and receiver efficiency
        n_rec[i] = receiver_eff_func(theta_r[i], rec_zleff, rec_b_0);
        
    
        # this is the position error, without taking max intensity into account -> useful for checking position error effects
        UA_F_disp = gaussNcone_transmission_func(r_aperture,r_b,d_ave,d[i],L_W)
        UA_n_pos[i] = position_eff_func(theta_r[i], pos_err, point_err, UA_F_disp, h, r); 
        
        # this function will determine the efficiency associated with the pointing and position error of the satellite
        # This also incorperates lost energy from changing apperent reciever size
        n_pos[i] = position_eff_func(theta_r[i], pos_err, point_err, F_disp[i,:,:], h, r);

        E_R[i] = t_step*P_T[i]*n_rec[i]*n_pos[i] # total energy recieved, per time step
        E_T[i] = t_step*P_T[i] # Total energy transmit, per time step
    
        
    E_R_tot = sum(E_R); # This is in Joules ->
    E_R_tot = E_R_tot*2.7778*10**-7 # kWh

    E_T_tot = sum(E_T); # This is in Joules ->
    E_T_tot = E_T_tot*2.7778*10**-7 # kWh

    Total_eff = E_R_tot/E_T_tot*100
    
    return [t_end, E_R_tot, Total_eff, Feasible]
    # transmission time, total recieved energy, total efficiency of power transmission
    

In [1]:


#print('Energy and Efficiency:')
#print('  Energy Received is',round(E_R_tot,3),'Wh');
#print('  Energy Transmitted is',round(E_T_tot,3),'Wh');
#print('  Transmission Energy Efficiency',round(E_R_tot/E_T_tot*100,2),'%\n');

#print('Maximum Transmission Times:')
#print('  Total Transmission time is', round(t_end,2),'s')
#print('    -Total time battery can transmit is', round(t_max_battery,2),'s')
#print('    -Total time receiver can receive is', round(t_max_receiver,2),'s\n')

#print('SC Attitude and Pointing Requirments:')
#print('  The necessary focal length of this ideal lens is',round(focal_length/1000,2), 'km')
#print('  The necessary maximum pointing speed is',round(max(dtheta_s_approx),5), 'rad\s')
#print('  The necessary maximum poingint acceleration is',round(max(ddtheta_s_approx),5), 'rad\s^2')



